<a href="https://colab.research.google.com/github/yueguo1997/SNA_recommender_system/blob/main/SNA_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch

In [42]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data preparation

In [43]:
cur_path = "/content/drive/My Drive/SNA final project/"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/SNA final project


In [44]:
# Reading in datasets/book1.csv
df = pd.read_csv('bonanza_buyer_item_seller_id_R_2.txt', sep='::', header=None, names=['buyer', 'item', 'seller', 'timestamp'])

# Printing out the head of the dataset
# split the data into train and test
train_data, test_data = train_test_split(df, test_size=0.2)

<ipython-input-44-90b1d1afe9ae>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('bonanza_buyer_item_seller_id_R_2.txt', sep='::', header=None, names=['buyer', 'item', 'seller', 'timestamp'])


# Create Graph

In [45]:
import networkx as nx
import csv

In [46]:
# give each node unique node id
nodes_type = ['buyer', 'item', 'seller']
index = 0

dictionary = {}
for t in nodes_type:
  for i in range(len(df)):
    if (df[t][i],t) not in dictionary:
      dictionary[(df[t][i],t)] = index
      index += 1




In [47]:
# Add nodes for buyers, items, and sellers to train graph
G_train = nx.MultiGraph()
G_test = nx.MultiGraph()

for t in nodes_type:
  for i in train_data[t].unique():
     G_train.add_node(dictionary[(i,t)], type=t, number = i)
  

In [48]:
# Add nodes for buyers, items, and sellers to test graph
for t in nodes_type:
  for i in test_data[t].unique():
     G_test.add_node(dictionary[(i,t)], type=t, number = i)

In [51]:
#Add edges to the train graph
for _, row in train_data.iterrows():
  buyer = int(row['buyer'])
  item = int(row['item'])
  seller = int(row['seller'])
  time = row['timestamp']
  G_train.add_edge(dictionary[(buyer,'buyer')], dictionary[(item,'item')], type='buys', timestamp = time)
  G_train.add_edge(dictionary[(item,'item')], dictionary[(seller,'seller')], type='sold_by')


In [52]:
#Add edges to the test graph
for _, row in test_data.iterrows():
  buyer = int(row['buyer'])
  item = int(row['item'])
  seller = int(row['seller'])
  time = row['timestamp']
  G_test.add_edge(dictionary[(buyer,'buyer')], dictionary[(item,'item')], type='buys', timestamp = time)
  G_test.add_edge(dictionary[(item,'item')], dictionary[(seller,'seller')], type='sold_by')


In [56]:
# generate a dictionary for the model result find the corresponding product and seller
output_d = {}
for i in dictionary:
  output_d[dictionary[i]] = i


In [58]:
# Print some statistics about the graph
print("Number of nodes on train:", G_train.number_of_nodes())
print("Number of edges on train:", G_train.number_of_edges())
print("Number of unique edge types on train:", len(set(nx.get_edge_attributes(G_train, 'type').values())))

print("Number of nodes on test:", G_test.number_of_nodes())
print("Number of edges on test:", G_test.number_of_edges())
print("Number of unique edge types on test:", len(set(nx.get_edge_attributes(G_test, 'type').values())))

Number of nodes on train: 14925
Number of edges on train: 62560
Number of unique edge types on train: 2
Number of nodes on test: 7625
Number of edges on test: 15640
Number of unique edge types on test: 2


In [66]:
# Check the number of different nodes
print("Number of seller nodes on train:", len([n for n, attr in G_train.nodes(data=True) if attr['type'] == 'seller']))
print("Number of buyer nodes on train:", len([n for n, attr in G_train.nodes(data=True) if attr['type'] == 'buyer']))
print("Number of item nodes on train:", len([n for n, attr in G_train.nodes(data=True) if attr['type'] == 'item']))
print("Number of seller nodes on test:", len([n for n, attr in G_test.nodes(data=True) if attr['type'] == 'seller']))
print("Number of buyer nodes on test:", len([n for n, attr in G_test.nodes(data=True) if attr['type'] == 'buyer']))
print("Number of item nodes on test:", len([n for n, attr in G_test.nodes(data=True) if attr['type'] == 'item']))

Number of seller nodes on train: 3686
Number of buyer nodes on train: 8355
Number of item nodes on train: 2884
Number of seller nodes on test: 1880
Number of buyer nodes on test: 4150
Number of item nodes on test: 1595


In [69]:
# Check the number of different nodes
print("Number of sold by edges on train:", len([n for n,v, attr in G_train.edges(data=True) if attr['type'] == 'sold_by']))
print("Number of buy edges on train:", len([n for n,vars, attr in G_train.edges(data=True) if attr['type'] == 'buys']))
print("Number of sold by edges  on test:", len([n for n,v, attr in G_test.edges(data=True) if attr['type'] == 'sold_by']))
print("Number of buy edges on test:", len([n for n,v, attr in G_test.edges(data=True) if attr['type'] == 'buys']))


Number of sold by edges on train: 31280
Number of buy edges on train: 31280
Number of sold by edges  on test: 7820
Number of buy edges on test: 7820


# Write graph data

In [72]:
nx.write_graphml(G_train, "train.graphml")

In [73]:
nx.write_graphml(G_test, "test.graphml")

In [75]:
G_train = nx.read_graphml("train.graphml")
G_test = nx.read_graphml("test.graphml")

In [76]:
print("Number of seller nodes on train:", len([n for n, attr in G_train.nodes(data=True) if attr['type'] == 'seller']))
print("Number of buyer nodes on train:", len([n for n, attr in G_train.nodes(data=True) if attr['type'] == 'buyer']))
print("Number of item nodes on train:", len([n for n, attr in G_train.nodes(data=True) if attr['type'] == 'item']))
print("Number of seller nodes on test:", len([n for n, attr in G_test.nodes(data=True) if attr['type'] == 'seller']))
print("Number of buyer nodes on test:", len([n for n, attr in G_test.nodes(data=True) if attr['type'] == 'buyer']))
print("Number of item nodes on test:", len([n for n, attr in G_test.nodes(data=True) if attr['type'] == 'item']))
print("Number of sold by edges on train:", len([n for n,v, attr in G_train.edges(data=True) if attr['type'] == 'sold_by']))
print("Number of buy edges on train:", len([n for n,vars, attr in G_train.edges(data=True) if attr['type'] == 'buys']))
print("Number of sold by edges  on test:", len([n for n,v, attr in G_test.edges(data=True) if attr['type'] == 'sold_by']))
print("Number of buy edges on test:", len([n for n,v, attr in G_test.edges(data=True) if attr['type'] == 'buys']))

Number of seller nodes on train: 3686
Number of buyer nodes on train: 8355
Number of item nodes on train: 2884
Number of seller nodes on test: 1880
Number of buyer nodes on test: 4150
Number of item nodes on test: 1595
Number of sold by edges on train: 31280
Number of buy edges on train: 31280
Number of sold by edges  on test: 7820
Number of buy edges on test: 7820


# wirte dictionary data

In [77]:
import json

In [83]:
df = pd.DataFrame(output_d)
df.to_csv('dictionary.csv')

,0,1,2,3,4,5,6,7,8,9,...,16087,16088,16089,16090,16091,16092,16093,16094,16095,16096
0,0,1,2,3,4,5,6,7,8,9,...,4046,4047,4048,4049,4050,4051,4052,4053,4054,4055
1,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,...,seller,seller,seller,seller,seller,seller,seller,seller,seller,seller
